**Name:** LI XINYAN

**EID:** 55670594

**Kaggle Team Name:** Saaries98

# CS5489 - Assignment 1 - Tweet Sentiment Analysis

## Final submission
In this file, put the code that generates your final Kaggle submission. It will be used to verify that your Kaggle submission is reproducible.

In [1]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
import csv
random.seed(100)

In [2]:
def read_text_data(fname):
    txtdata = []
    classes = []
    topics  = []
    with open(fname, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            # get the text
            txtdata.append(row[0])
            # get the class (convert to integer)
            if len(row)>1:
                classes.append(row[1])
                topics.append(row[2])
    
    if (len(classes)>0) and (len(txtdata) != len(classes)):        
        raise Exception("mismatched length!")
    
    return (txtdata, classes, topics)

def write_csv_kaggle_sub(fname, Y):
    # fname = file name
    # Y is a list/array with class entries
    
    # header
    tmp = [['Id', 'Prediction']]
    
    # add ID numbers for each Y
    for (i,y) in enumerate(Y):
        tmp2 = [(i+1), y]
        tmp.append(tmp2)
        
    # write CSV file
    with open(fname, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(tmp)

In [3]:
# load the data
# (if using Kaggle notebooks you need to include the directory path: /kaggle/input/cs5489-2020b-assignment-1/)
(traintxt, trainY, traintopic) = read_text_data("sanders_tweets_train.txt")
(testtxt, _, _)                = read_text_data("sanders_tweets_test.txt")

print(len(traintxt))
print(len(testtxt))

2396
1028


In [4]:
# write your predictions on the test set
classnames = unique(trainY)
print(classnames)
i = random.randint(len(classnames), size=len(testtxt))
predY = classnames[i]
# write_csv_kaggle_sub("my_submission.csv", predY)

['negative' 'neutral' 'positive']


## Helper Functions

In [5]:
import re
from afinn import Afinn
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

def eva_afinn_score(string):
    af = Afinn(emoticons=True)
    scr = af.score(string)
    return scr

def eva_blob_score(string):
    scr = TextBlob(string).sentiment.polarity
    return scr

def add_extra_feature(df, tweet_column):
    df['number_of_quote'] = tweet_column.apply(lambda x: (len(re.findall(r'"', x))/2))
    df['number_of_stop'] = tweet_column.apply(lambda x: (len(re.findall(r'.', x))))
    df['number_of_exclamation'] = tweet_column.apply(lambda x: (len(re.findall(r'!', x))))
    df['number_of_questionmark'] = tweet_column.apply(lambda x: (len(re.findall(r'[?]', x))))
    df['number_of_hashtag'] = tweet_column.apply(lambda x: (len(re.findall(r'#', x))))
    df['number_of_mention'] = tweet_column.apply(lambda x: (len(re.findall(r'@', x))))
    df['number_of_comma'] = tweet_column.apply(lambda x: (len(re.findall(r',', x))))

def eva_comp(df, tweet_column):
    df['apple'] = tweet_column.apply(lambda x: int('apple' in x))
    df['google'] = tweet_column.apply(lambda x: int('google' in x))
    df['twitter'] = tweet_column.apply(lambda x: int('twitter' in x))
    df['microsoft'] = tweet_column.apply(lambda x: int('microsoft' in x))

def clean_str(string):
    string = re.sub("&lt;3","love", string)
    string = re.sub("http://[\w|./]*","", string)
    string = re.sub(r"[^A-Za-z]", " ", string)
    return string.strip().lower()

def tokenize(text):
    return word_tokenize(text)

def stemming(words):
    ps = PorterStemmer()
    stem_words = []
    for w in words:
        w = ps.stem(w)
        stem_words.append(w)
    return stem_words

def word_to_sen(words):
    sen = ''
    for w in words:
        sen = sen + w + ' '
    return sen

# Data pre-processing

In [6]:
import pandas as pd
import numpy as np

c1 = np.array(traintxt)
c2 = np.array(traintopic)
c3 = np.array(trainY)

dix = {"traintxt": c1,
       "traintopic":c2,
       "trainY":c3}

train_data = pd.DataFrame(dix)

In [7]:
# Helper function: 1~3
add_extra_feature(train_data, train_data["traintxt"])
train_data['affin'] = train_data['traintxt'].apply(eva_afinn_score)
train_data['blob'] = train_data['traintxt'].apply(eva_blob_score)

# Helper function: 5
train_data['traintxt'] = train_data['traintxt'].apply(clean_str)

# Helper function: 4
eva_comp(train_data, train_data["traintxt"])

# Helper function: 6~8
train_data["traintxt"] = train_data["traintxt"].apply(tokenize)
train_data["traintxt"] = train_data["traintxt"].apply(stemming)
train_data["traintxt"] = train_data["traintxt"].apply(word_to_sen)

# The tweets length might also be helpeful
train_data['length'] =  train_data["traintxt"].apply(lambda x: (len(x)))

In [8]:
# Ptint pre-processing results
display(train_data)

,traintxt,traintopic,trainY,number_of_quote,number_of_stop,number_of_exclamation,number_of_questionmark,number_of_hashtag,number_of_mention,number_of_comma,affin,blob,apple,google,twitter,microsoft,length
0,forget the phone nice ui like the scroll featu...,google,positive,0.0,98,0,0,3,0,0,4.0,0.600000,0,1,0,0,67
1,i hate when my phne do what it want on twitter,twitter,neutral,0.0,47,0,0,1,0,0,-2.0,-0.800000,0,0,1,0,47
2,and onli the first imag in my photo roll made ...,apple,negative,0.0,113,0,2,0,1,2,-4.0,-0.056667,1,0,0,0,99
3,the lock screen now ha facial recognit capabl ...,google,neutral,0.0,76,1,0,3,0,0,3.0,0.000000,0,1,0,0,63
4,teamgooglenexu rt b y googl samsung perfect ic...,google,neutral,0.0,86,0,0,5,1,0,3.0,1.000000,0,1,0,0,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2391,karth vader when did you chang your nick again...,twitter,neutral,0.0,131,0,2,1,1,0,-3.0,-0.500000,0,0,1,0,120
2392,goodnight twitter,twitter,neutral,0.0,20,0,0,1,0,0,2.0,0.500000,0,0,1,0,18
2393,androidpolic duli note i hope aosp is updat wi...,google,neutral,0.0,82,0,0,2,1,0,2.0,0.000000,0,1,0,0,66
2394,microsoft s plan for bring it bi tool to io an...,microsoft,neutral,0.0,128,0,0,2,0,2,0.0,0.500000,0,0,0,1,82


# Feature Encoding

### Use BoW representation

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import preprocessing

comp_vectorizer = CountVectorizer(max_features=3000, stop_words="english")
train_bow_features = comp_vectorizer.fit_transform(train_data["traintxt"]).toarray()
train_bow = pd.DataFrame(train_bow_features)

In [10]:
display(train_bow)

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2391,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2392,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2393,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2394,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Train Classifier

In [11]:
# Consider class unbalanced
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

selected_features = ['number_of_exclamation', 'number_of_questionmark', 'number_of_hashtag',
                     'number_of_quote', 'number_of_stop', 'number_of_mention',
                     'affin', 'blob', 'apple', 'google', 'microsoft']

train = pd.merge(train_bow, train_data[selected_features], left_index=True, right_index=True)
train_X, test_X, train_Y, test_Y = train_test_split(train, trainY, test_size=0.2)

bow_rf_clf_model = RandomForestClassifier(n_estimators = 1000, min_samples_split = 10, bootstrap = True,
                                          max_depth = 100, random_state = 0, class_weight='balanced')

bow_rf_clf_model.fit(train_X, train_Y)
pred_Y = bow_rf_clf_model.predict(test_X)

print("Accuracy => ", accuracy_score(pred_Y, test_Y))
print("\nRandom Forest Classifier results:")
print(classification_report(test_Y, pred_Y))

Accuracy =>  0.81875

Random Forest Classifier results:
              precision    recall  f1-score   support

    negative       0.72      0.74      0.73        72
     neutral       0.86      0.91      0.88       333
    positive       0.71      0.49      0.58        75

    accuracy                           0.82       480
   macro avg       0.76      0.71      0.73       480
weighted avg       0.81      0.82      0.81       480



# Predict data
### Prediction of the best classifier

In [12]:
# testtxt
c1 = np.array(testtxt)
dix = {"testtxt": c1}
test_data = pd.DataFrame(dix)

add_extra_feature(test_data, test_data["testtxt"])

test_data['affin'] = test_data['testtxt'].apply(eva_afinn_score)
test_data['blob'] = test_data['testtxt'].apply(eva_blob_score)
test_data['traintxt'] = test_data['testtxt'].apply(clean_str)

eva_comp(test_data, test_data["testtxt"])
test_data['length'] =  test_data["testtxt"].apply(lambda x: (len(x)))

test_data["testtxt"] = test_data["testtxt"].apply(tokenize)
test_data["testtxt"] = test_data["testtxt"].apply(stemming)
test_data["testtxt"] = test_data["testtxt"].apply(word_to_sen)

In [13]:
test_bow_features = comp_vectorizer.transform(test_data["testtxt"]).toarray()
test_bow = pd.DataFrame(test_bow_features)

In [14]:
selected_features = ['number_of_exclamation', 'number_of_questionmark', 'number_of_hashtag',
                     'number_of_quote', 'number_of_stop', 'number_of_mention',
                     'affin', 'blob', 'apple', 'google', 'microsoft']

test = pd.merge(test_bow, test_data[selected_features], left_index=True, right_index=True)
pred_Y = bow_rf_clf_model.predict(test)

print(pred_Y)
print(len(pred_Y))

['neutral' 'neutral' 'neutral' ... 'neutral' 'neutral' 'neutral']
1028


In [15]:
write_csv_kaggle_sub("my_submission.csv", pred_Y)

# Closing Remarks

**The best performance my model can achieve is balanced RandomForest classifier + meta features + BoW (This combination actually surprise me). It gives me the following hints:**

1. BoW can outperform TFIDF
2. Data pre-processing and cleansing is very important
3. Analyae the database before taking any actions
4. Meta features are very helpful, try your best to make use of it
5. The LSTM NN I trained above seems overfitted, it acts poor on the test data

**Before doing this assignment, I alse found some useful blogs that verified my ideas:**
1. How to use BERT and Google's word embedding: https://www.kaggle.com/gunesevitan/nlp-with-disaster-tweets-eda-cleaning-and-bert#6.-Cross-validation
2. How to analyse your database: https://towardsdatascience.com/text-analysis-feature-engineering-with-nlp-502d6ea9225d
3. Feature Engineering (meta feature extraction, fantastic): https://www.analyticsvidhya.com/blog/2021/04/a-guide-to-feature-engineering-in-nlp/
4. How to analyse your database: https://www.analyticsvidhya.com/blog/2021/06/twitter-sentiment-analysis-a-nlp-use-case-for-beginners/

**My special tricks summary:**
1. Handle special emoji. For example, '&lt;3' is the HTML code for a heart and suggests positive sentiment
2. Extract meta features in punctuations. It's intuitive that people will use '!!!', '....', '???' to express special sentiments
3. Make use of public sentiment analysis tools like Afinn and TextBlob
4. Be careful about the class unbalance will be really helpful (raise 2% acc)